# LimberJack.jl Tutorial

## Basic Usage

The core of  $\texttt{LimberJack.jl}$ is the $\texttt{Cosmology}$ structure and its homonymous constructor. $\texttt{Cosmology}$'s role is to contain all the information on how to compute the theoretical predictions. When the \texttt{Cosmology} structure is initiated it computes theoretical predictions for the expansion history, comoving distance, the growth factor and the matter power spectrum according to the provided prescriptions. The computation of background quantities occurs within $\texttt{core.jl}$ itself. However, the computation of the matter power spectrum takes place across three different modules, $\texttt{boltzmann.jl}$, $\texttt{growth.jl}$ and $\texttt{halofit.jl}$, which compute the primordial matter power spectrum, the linear growth factor and the non-linear corrections respectively. The different predictions are evaluated at a grid of values which are then used build interpolators for each of the respective quantities. The interpolators are then stored inside the $\texttt{Cosmology}$ structure. This allows the user to quickly compute the theoretical predictions at arbitrary values by using a series of public functions. These functions take the $\texttt{Cosmology}$ structure and the necessary inputs. Inside these functions the corresponding interpolator is evaluated at the provided inputs to return the prediction to the user. 

In [ ]:
# Activate env
using Pkg
Pkg.activate(".")
# Imports
using LimberJack

# create LimberJack.jl Cosmology instance
cosmo = Cosmology() 
# same as 
# cosmology = Cosmology(|$\Omega_m=0.3$|, |$\Omega_b=0.05$|,
#                       h=0.67, ns=0.96, |$\sigma_8=0.81$|)     
 
zs = [0.1, 0.5, 1.0, 3.0]
H = cosmo.cpar.h*100*Ez(cosmo, zs)
chi = comoving_radial_distance(cosmo, zs)
Dz = growth_factor(cosmo, zs)
fz = growth_rate(cosmo, zs)
fs8z = fs8(cosmo, zs);

Computing the matter power spectrum is just as easy. However, the computation is subject to a series of options that the user can alter. By default $\texttt{Cosmology}$ will use the E\&H formula to find the matter power spectrum and it will not apply non-linear corrections.  These settings can be changed by specifying the keyword arguments $\texttt{tk\_mode}$ and $\texttt{pk\_mode}$ which control the used transfer function and whether non-linear corrections are applied to the power spectrum respectively. In terms transfer function $\texttt{LimberJack}$ offers two possibilities  $\texttt{tk\_mode}$ = $\texttt{EisHu}$ (default) / $\texttt{EmuPk}$ which correspond to using the Eisenstein and Hu formula or $\texttt{:EmuPk}$. Similarly, $\texttt{pk\_mode}$ = $\texttt{:linear}$ / $\texttt{:Halofit}$ which determines whether or not non-linear corrections are applied using $\texttt{Halofit}$. $\texttt{LimberJack.jl}$ offers two distinct public functions to evaluate either the linear or non-linear matter power spectrum regardless of the choice in $\texttt{pk\_mode}$. However, if $\texttt{pk\_mode}$ = $\texttt{:linear}$ the two functions will return the linear matter power spectrum.

In [ ]:
cosmo_lin_EisHu = Cosmology(Omega_m=0.25, Omega_b=0.03,
                             h=0.70, ns=1.0,
                             sigma_8=0.78,
                             tk\_mode=:EisHu,
                             pk\_mode=:linear) 
                             
cosmo_nonlin_EisHu = Cosmology(Omega_m=0.25, Omega_b=0.03,
                             h=0.70, ns=1.0,
                             sigma_8=0.78,
                             tk\_mode=:EisHu,
                             pk\_mode=:Halofit) 

cosmo_nonlin_emupk = Cosmology(Omega_m=0.25, Omega_b=0.03,
                             h=0.70, ns=1.0,
                             sigma_8=0.78,
                             tk\_mode=:EmuPk,
                             pk\_mode=:Halofit)

zs = [0.1, 0.3, 0.5]
ks = [100, 300, 1000]
lin_eh_Pks = lin_Pk(cosmo_lin_EisHu, ks, zs) = nonlin_Pk(cosmo_lin_EisHu, ks, zs)
nonlin_eh_Pks = nonlin_Pk(cosmo_nonlin_EisHu, ks, zs)
nonlin_emupk_Pks = nonlin_Pk(cosmo_nonlin_emupk, ks, zs)

Computing angular power spectra is a slightly more involved process. An example of how this is done in $\texttt{LimberJack}$ can be found below. In this code we can see that first a $\texttt{Cosmology}$ structure must be initiated. The $\texttt{Cosmology}$ structure automatically computes the matter power spectrum given the user specifications. Then the user must compute the distance kernels of the relevant tracers by proving the corresponding  $\texttt{LimberJack}$ public functions with the $\texttt{Cosmology}$ structure and the distribution of sources. Moreover, different tracers can be impacted by different systematic effects. These systematics are accounted by incorporating a series of nuisance parameters that can also be provided to the tracers public functions of  $\texttt{LimberJack}$. The output of the tracer functions is a $\texttt{Tracer}$ structure that hosts an interpolator for the corresponding distance kernel and the corrections. The angular power spectra can computed by providing the $\texttt{AngularCls}$ public function with the aforementioned $\texttt{Cosmology}$ and $\texttt{Tracer}$ objects as well as the desired multipoles.

In [ ]:
# Initiate cosmology
cosmo = Cosmology()

# Define a distribution of sources
z = Vector(range(0., stop=2., length=256))
nz = @. exp(-0.5*((z-0.5)/0.05)^2)

# Create tracer objects
bias = 1.0 # Galaxy-mass bias
tg = NumberCountsTracer(cosmo, z, nz; b=bias)

mbias = 0.0 # shape multiplicative bias
A_IA = 0.0 # Amplitude of intrinsic alignments power spectrum
alpha_IA = 0.0 # Slope of intrinsic alignments power spectrum
ts = WeakLensingTracer(cosmo, z, nz;
                       m=mbias, IA_params=[A_IA, alpha_IA])

tk = CMBLensingTracer(cosmo)

# Compute power spectra
ls = [10.0, 30.0, 100.0, 300.0, 1000.0]
Cl_gg = angularCℓs(cosmo, tg, tg, ls)
Cl_gs = angularCℓs(cosmo, tg, ts, ls)
Cl_ss = angularCℓs(cosmo, ts, ts, ls)
Cl_gk = angularCℓs(cosmo, tg, tk, ls)
Cl_sk = angularCℓs(cosmo, ts, tk, ls);

## Statistical Inference

Finally, we show how to use $\texttt{Turing}$ in unison with $\texttt{LimberJack}$ to build and sample an statistical model for a the DESY1 3x2 analysis. The first step is to load the data. For this purpose we will use the libraries $\texttt{YAML}$ and $\texttt{sacc}$, ubiquitous in astrophysics. $\texttt{Julia}$ counts with a native implementation of $\texttt{YAML}$ but not of $\texttt{sacc}$. However, calling $\texttt{Python}$ libraries from $\texttt{Julia}$ is extremely simple thanks to the $\texttt{PythonCall.jl}$ library.  $\texttt{PythonCall.jl}$ allows us to import $\texttt{sacc.py}$ as a $\texttt{Julia}$ module and read files entirely within $\texttt{Julia}$. Note that in order to this we must first install $\texttt{sacc}$ in the $\texttt{Python}$ environment of $\texttt{LimberJack.jl}$ or point $\texttt{PythonCall}$ to our local $\texttt{Pytthon}$ installation. Instructions on how to do this can be found in the $\texttt{LimberJack}$ GitHub.  Once the data are loaded they must be passed to the $\texttt{LimberJack.jl}$ public function $\texttt{make\_data}$ which turns the files into $\texttt{Julia}$ structures that $\texttt{LimberJack.jl}$ can easily manage.

After that, the user can use $\texttt{Turing.jl}$'s $\texttt{@model}$ macro to define an statistical model. Inside the model, the user must define the priors for the parameters of the model using $\texttt{Distributions.jl}$. Note that while the cosmology parameters can be directly passed to the $\texttt{Cosmology}$ structure constructor the nuisance parameters must be stored inside a $\texttt{Julia}$ dictionary. The name of these parameters inside the dictionary must follow a strict convention "$\texttt{tracer\_name}$ + __ + $\texttt{bin\_number}$ + _ + $\texttt{nuisance\_parameter\_name}$".

In order to obtain the theoretical prediction for the DESY1 3x2 analysis data vector the user must provide the just initiated $\texttt{Cosmology}$ structure and the $\texttt{meta}$ and $\texttt{files}$ structures generated by $\texttt{make\_data}$ to the public function $\texttt{Theory}$. Moreover, the user must also provide the nuisance parameter dictionary using the $\texttt{Nuisances}$ keyword argument. The $\texttt{Theory}$ function orchestrates the computation of the theory vector, computing the necessary distance kernels and evaluating the angular power spectra in the correct order. 

Once a theoretical prediction has been obtained, the user must define how the data is distributed with respect to the theory prediction. In the case of a Gaussian likelihood, this corresponds to a multivariate Gaussian distribution with mean the theory prediction and covariance matrix the data covariance matrix. All is left is to do is to use $\texttt{Turing.jl}$ to condition the model on the observations , define the sampler we wish to use and sample the model. For a more thorough explanation of how to use $\texttt{Turing.jl}$ and its different options please see $\texttt{Turing.jl}$'s documentation. 

In [ ]:
# Imports
using LinearAlgebra
using Turing
using LimberJack
using YAML
using PythonCall
sacc = pyimport("sacc");

# Load data
sacc_path = "cls_FD_covG.fits"
yaml_path = "DESY1.yml"
sacc_file = sacc.Sacc().load_fits(sacc_path)
yaml_file = YAML.load_file(yaml_path)
meta, files = make_data(sacc_file, yaml_file)
data = meta.data
cov = meta.cov

# Define model
@model function model(data;
    meta=meta, 
    files=files)
    Ωm ~ Uniform(0.2, 0.6)
    Ωb ~ Uniform(0.028, 0.065)
    h ~ TruncatedNormal(0.72, 0.05, 0.64, 0.82)
    σ8 ~ Uniform(0.4, 1.2)
    ns ~ Uniform(0.84, 1.1)

    DESgc__0_b ~ Uniform(0.8, 3.0)
    DESgc__1_b ~ Uniform(0.8, 3.0)
    DESgc__2_b ~ Uniform(0.8, 3.0)
    DESgc__3_b ~ Uniform(0.8, 3.0)
    DESgc__4_b ~ Uniform(0.8, 3.0)
    DESgc__0_dz ~ TruncatedNormal(0.0, 0.007, -0.2, 0.2)
    DESgc__1_dz ~ TruncatedNormal(0.0, 0.007, -0.2, 0.2)
    DESgc__2_dz ~ TruncatedNormal(0.0, 0.006, -0.2, 0.2)
    DESgc__3_dz ~ TruncatedNormal(0.0, 0.01, -0.2, 0.2)
    DESgc__4_dz ~ TruncatedNormal(0.0, 0.01, -0.2, 0.2)
    DESwl__0_dz ~ TruncatedNormal(-0.001, 0.016, -0.2, 0.2)
    DESwl__1_dz ~ TruncatedNormal(-0.019, 0.013, -0.2, 0.2)
    DESwl__2_dz ~ TruncatedNormal(0.009, 0.011, -0.2, 0.2)
    DESwl__3_dz ~ TruncatedNormal(-0.018, 0.022, -0.2, 0.2)
    DESwl__0_m ~ Normal(0.012, 0.023)
    DESwl__1_m ~ Normal(0.012, 0.023)
    DESwl__2_m ~ Normal(0.012, 0.023)
    DESwl__3_m ~ Normal(0.012, 0.023)
    A_IA ~ Uniform(-5, 5) 
    alpha_IA ~ Uniform(-5, 5)

    nuisances = Dict("DESgc__0_b" => DESgc__0_b,
                     "DESgc__1_b" => DESgc__1_b,
                     "DESgc__2_b" => DESgc__2_b,
                     "DESgc__3_b" => DESgc__3_b,
                     "DESgc__4_b" => DESgc__4_b,
                     "DESgc__0_dz" => DESgc__0_dz,
                     "DESgc__1_dz" => DESgc__1_dz,
                     "DESgc__2_dz" => DESgc__2_dz,
                     "DESgc__3_dz" => DESgc__3_dz,
                     "DESgc__4_dz" => DESgc__4_dz,
                     "DESwl__0_dz" => DESwl__0_dz,
                     "DESwl__1_dz" => DESwl__1_dz,
                     "DESwl__2_dz" => DESwl__2_dz,
                     "DESwl__3_dz" => DESwl__3_dz,
                     "DESwl__0_m" => DESwl__0_m,
                     "DESwl__1_m" => DESwl__1_m,
                     "DESwl__2_m" => DESwl__2_m,
                     "DESwl__3_m" => DESwl__3_m,
                     "A_IA" => A_IA,
                     "alpha_IA" => alpha_IA,)

    cosmology = Cosmology(Ωm= Ωm,  Ωb=Ωb, h=h, ns=ns, σ8=σ8,
                          tk_mode=:EisHu,
                          pk_mode=:Halofit)

    theory = Theory(cosmology, meta, files; Nuisances=nuisances)
    data ~ MvNormal(theory, cov)
end

# Condition model on data
cond_model = model(data)

# Define sampler
nadapts = 500
TAP = 0.65
sampler = NUTS(adaptation, TAP)

# Sample model 
iterations = 1000
chain = sample(cond_model, sampler, iterations) 

## Plotting

In order to plot the results I use ```GetDist.py``` as follows:

```python
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd
import os
import getdist
from getdist import plots, MCSamples
import sacc

def add_chains(path):
    chains = []
    i = 1 
    while os.path.isfile(path+"chain_{}.csv".format(i)):
        chain = pd.read_csv(path+"chain_{}.csv".format(i))
        chains.append(chain)
        i += 1
    return pd.concat(chains)

test = add_chains("../chains/test_TAP_0.65/")

labels_dict = {'eta': '\eta',
               'l': 'l',
               'h': 'h',
               'Ωm': '\Omega_m',
               'Ωb': '\Omega_b',
               'ns': 'n_s',
               's8': '\sigma_8',
               'S8': 'S_8',
               'A_IA': 'A_{IA}',
               'alpha_IA': r'\alpha_{IA}',
               'DESgc__0_0_dz': 'dz_{DESY1gc \, 0}',
               
               'DESgc__1_0_dz': 'dz_{DESY1gc \, 1}',
               'DESgc__2_0_dz': 'dz_{DESY1gc \, 2}',
               'DESgc__3_0_dz': 'dz_{DESY1gc \, 3}',
               'DESgc__4_0_dz': 'dz_{DESY1gc \, 4}',
               
               'DESwl__0_e_dz': 'dz_{DESY1wl \, 0}',
               'DESwl__1_e_dz': 'dz_{DESY1wl \, 1}',
               'DESwl__2_e_dz': 'dz_{DESY1wl \, 2}',
               'DESwl__3_e_dz': 'dz_{DESY1wl \, 3}',
               
               'DESgc__0_0_b': 'b_{DESY1 \, 0}',
               'DESgc__1_0_b': 'b_{DESY1 \, 1}',
               'DESgc__2_0_b': 'b_{DESY1 \, 2}',
               'DESgc__3_0_b': 'b_{DESY1 \, 3}',
               'DESgc__4_0_b': 'b_{DESY1 \, 4}',
               
               'DESwl__0_e_m': 'm_{DESY1 \, 0 }',
               'DESwl__1_e_m': 'm_{DESY1 \, 1 }',
               'DESwl__2_e_m': 'm_{DESY1 \, 2 }', 
               'DESwl__3_e_m': 'm_{DESY1 \, 3 }',
               
               'eBOSS__0_0_b': 'b_{eBOSS \, 0}',
               'eBOSS__1_0_b': 'b_{eBOSS \, 1}',
               
               "DECALS__0_0_b": 'b_{DECALS \, 0}',
               "DECALS__1_0_b": 'b_{DECALS \, 1}',
               "DECALS__2_0_b": 'b_{DECALS \, 2}',
               "DECALS__3_0_b": 'b_{DECALS \, 3}',
               
               "DECALS__0_0_dz": 'dz_{DECALS \, 0}',
               "DECALS__1_0_dz": 'dz_{DECALS \, 1}',
               "DECALS__2_0_dz": 'dz_{DECALS \, 2}',
               "DECALS__3_0_dz": 'dz_{DECALS \, 3}',

               "KiDS1000__0_e_dz": 'dz_{KiDS \, 0}',
               "KiDS1000__1_e_dz": 'dz_{KiDS \, 1}',
               "KiDS1000__2_e_dz": 'dz_{KiDS \, 2}',
               "KiDS1000__3_e_dz": 'dz_{KiDS \, 3}',
               "KiDS1000__4_e_dz": 'dz_{KiDS \, 4}',
               
               "KiDS1000__0_e_m": 'm_{KiDS \, 0}',
               "KiDS1000__1_e_m": 'm_{KiDS \, 1}',
               "KiDS1000__2_e_m": 'm_{KiDS \, 2}',
               "KiDS1000__3_e_m": 'm_{KiDS \, 3}',
               "KiDS1000__4_e_m": 'm_{KiDS \, 4}',
               
               "v[1]": "v_{1}", "v[2]": "v_{2}",
               "v[3]": "v_{3}", "v[4]": "v_{4}",
               "v[5]": "v_{5}", "v[6]": "v_{6}",
               "v[7]": "v_{7}", "v[8]": "v_{8}", 
               "v[9]": "v_{9}", "v[10]": "v_{10}",
               "v[11]": "v_{11}"}

def make_chain(file, label, ranges=dict({})):
    params = np.array(list(file.keys()))
    names = []
    labels = []
    samples = []
    print()
    print(label)
    for param in params:
        print(param)
        if param in labels_dict.keys():
            print(param)
            names.append(param) 
            labels.append(labels_dict[param]) 
            samples.append(file[param])
    if ('s8' in params) & ('Ωm' in params):
        print('S8')
        names.append('S8')
        labels.append(labels_dict['S8'])
        samples.append(file['s8']*np.sqrt(file['Ωm']/0.3))

    names = np.array(names)
    labels = np.array(labels)
    samples = np.transpose(np.array(samples))
    print("========")

    return MCSamples(samples=samples, names=names, labels=labels, label=label, ranges=ranges,  
                    settings={'mult_bias_correction_order':0,'smooth_scale_2D':0.4, 'smooth_scale_1D':0.3})

test_samples = make_chain(test, "Test");

g = plots.getSubplotPlotter(subplot_size=4)
g.triangle_plot(test_samples, filled=True)
```